# Annotating data with canapy - tutorial

With Canapy you can train an AI model to annotate bird songs. This tutorial shows you how to use this trained model to annotate data.

## Import libraries

In [1]:
from canapy.corpus import Corpus
from canapy.annotator import get_annotator
from config import default_config
from canapy import metrics

## Create Annotators

First, you have to create an Annotator object, there is three types possible:

In [2]:
syn_annotator = get_annotator("syn-esn")(default_config, "./tuto/spec")
nsyn_annotator = get_annotator("nsyn-esn")(default_config, "./tuto/spec")
ensemble_annotator = get_annotator("ensemble")(default_config, "./tuto/spec")

It has been given to all of them the `default_config`, but you can create your own configuration and applied it on your annotators.


## Import Data

Then you need to import an annotated corpus to train annotators, and a corpus that you want to annotate :

In [3]:
corpus_annotated_songs = Corpus.from_directory(
    audio_directory="/home/nathan/Documents/Code/canapy-test/data_short",
    annots_directory="/home/nathan/Documents/Code/canapy-test/data_short"
)

corpus_non_annotated_songs = Corpus.from_directory(
    audio_directory="/home/nathan/Documents/Code/canapy-test/data_short"
)

In [4]:
len(corpus_annotated_songs)

18

In this example, the `/tuto/annotated_songs` directory contains some .wav audio files and a .csv annotation files for each whereas `/tuto/non_annotated_songs` directory contains only .wav audio files, one per song, ready to be annotated.

The Corpus object stores the dataset in the form of a Pandas Dataframe, in the last case only paths to audio files, but can also store annotations as in the first case.

## Train Annotators

Every annotator needs to be trained before any manipulation :

In [5]:
syn_annotator.fit(corpus_annotated_songs)
nsyn_annotator.fit(corpus_annotated_songs)
ensemble_annotator.fit(corpus_annotated_songs)

INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
INFO:canapy:Applying corpus transform sort_annotations on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform merge_labels on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform sort_annotations on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform tag_silences on ((<Corpus at (audio) /

Fitting node ESN-0...


INFO:canapy:Applying corpus transform sort_annotations on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform merge_labels on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform sort_annotations on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform tag_silences on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'redo': False}).
INFO:canapy:Applying corpus transform sort_annotations on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/n

Fitting node ESN-1...


## Annotate New Songs

Syntaxic (`syn-esn`) and NonSyntaxic (`nsyn-esn`) Annotators annotate new songs with the following lines :

In [6]:
corpus_syn_predicted = syn_annotator.predict(corpus_non_annotated_songs)
corpus_nsyn_predicted = nsyn_annotator.predict(corpus_non_annotated_songs)

INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) None.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
Running ESN-0: 100%|███████████████████████████| 18/18 [00:00<00:00, 155.82it/s]
INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) None.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
Running ESN-1: 100%|███████████████████████████| 18/18 [00:00<00:00, 140.51it/s]


The case of ensemble is a little bit more complicated, because you need to predict some corpus using `return_raw=True`, and then predict with the results.

In [7]:
corpus_syn_predicted_raw = syn_annotator.predict(corpus_non_annotated_songs, return_raw=True)
corpus_nsyn_predicted_raw = nsyn_annotator.predict(corpus_non_annotated_songs, return_raw=True)

corpus_ensemble_predicted = ensemble_annotator.predict([corpus_syn_predicted_raw, corpus_nsyn_predicted_raw])

INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) None.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
Running ESN-0: 100%|███████████████████████████| 18/18 [00:00<00:00, 134.20it/s]
INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) None.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
Running ESN-1: 100%|███████████████████████████| 18/18 [00:00<00:00, 130.27it/s]


Note : if you planned to predict a corpus with the three types of annotators, no need to predict two times a corpus with syn and nsyn annotators (with and without return_raw) because return_raw just add data on the same corpus that is predicted when it is false.

## Store Annotations

Now that your annotators have made some annotions, you can store it on the disk :

In [8]:
corpus_syn_predicted.to_directory("./tuto/results/syn")
corpus_nsyn_predicted.to_directory("./tuto/results/nsyn")

<Corpus at (audio) None | (annots) None.>

That's it! You have now new annotation files !

## Analyze Annotators' Performances

You have annotated new songs, but how good the predictions are ?
To figure it out you should use `metrics` to compare annotations that you have made and those made by ESNs.

We will analyze only a Syntactic Annotator but the method remains the same for all of them.

In [9]:
corpus_annotated_songs_predicted = syn_annotator.predict(corpus_annotated_songs)

print(metrics.sklearn_confusion_matrix(corpus_annotated_songs, corpus_annotated_songs_predicted))

print(metrics.sklearn_classification_report(corpus_annotated_songs, corpus_annotated_songs_predicted))

print(metrics.segment_error_rate(corpus_annotated_songs, corpus_annotated_songs_predicted))


INFO:canapy:Applying audio transform compute_mfcc on ((<Corpus at (audio) /home/nathan/Documents/Code/canapy-test/data_short | (annots) /home/nathan/Documents/Code/canapy-test/data_short.>,), {'output_directory': PosixPath('tuto/spec'), 'resource_name': 'syn_mfcc', 'redo': False}).
Running ESN-0: 100%|███████████████████████████| 18/18 [00:00<00:00, 133.15it/s]


[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.96688742e-01 0.00000000e+00 3.31125828e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.29660708e-04 3.64830354e-03 9.61510398e-01 3.15578256e-02
  1.09449106e-03 5.47245531e-04 7.29660708e-04 1.82415177e-04]
 [0.00000000e+00 0.00000000e+00 6.23742455e-02 9.37625755e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.68539326e-02 8.42696629e-02
  8.98876404e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.76073620e-02 9.20245399e-03
  0.00000000e+00 9.63190184e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.26392252e-03 9.68523002e-03
  0.00000000e+00 0.00000000e+00 9.83050847e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]